In [1]:
import numpy as np 
from keras.layers import Input, Dense 
from keras.models import Model
from keras.datasets import mnist

In [67]:
(x_train, _), (x_test, _) = mnist.load_data()

In [79]:

x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(len(x_train), np.prod(x_train.shape[1:]))

x_test = x_test.reshape(len(x_test), np.prod(x_test.shape[1:]))


input_img = Input(shape=(784,))


encoded = Dense(32, activation='relu')(input_img) 


decoded = Dense(784, activation='sigmoid')(encoded)


autoencoder = Model(input_img, decoded)


encoded_input = Input(shape=(32,))

encoder = Model(input_img, encoded)


decoder_layer = autoencoder.layers[-1]

decoder = Model(encoded_input, decoder_layer(encoded_input))


In [69]:
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
history = autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
encoded_imgs = encoder.predict(x_test) 
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
import matplotlib.pyplot as plt
n = 10  
plt.figure(figsize=(20, 4))
for i in range(n):

    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [77]:
mse = np.mean(np.power(x_test - decoded_imgs, 2), axis=1)
error_threshold = 2 * mse.mean() 

In [ ]:
anomalies = mse > error_threshold
print("Found %d anomalies!" % np.sum(anomalies))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()